### 지도학습 기반 감성 분석 실습 – IMDB 영화평

In [2]:
import pandas as pd

review_df = pd.read_csv('./data/labeledTrainData.tsv', header=0, sep="\t", quoting=3)
review_df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


* id: 리뷰글 id
* review: 영화 리뷰 text
* sentiment: 1 => 긍정, 0 => 부정

* 리뷰열의 하나의 데이터를 확인 (자연어가 어떤 텍스트인지 확인하는 용도)

In [3]:
review_df['review'][0]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

* 한글 번역 결과
  최근 MJ와 관련된 일들이 일어나면서 저는 그의 음악을 듣기 시작했고, 가끔씩 다큐멘터리를 보고, 'The Wiz'와 'Moonwalker'를 다시 봤습니다. 아마도 80년대에 정말 멋있다고 생각했던 이 사람에 대해 뭔가 통찰을 얻고 싶어서 그가 유죄인지 무죄인지 스스로 결정하고 싶은 것 같습니다. 'Moonwalker'는 부분적으로는 전기 영화이고 부분적으로는 피처 필름인데, 원래 개봉했을 때 극장에서 본 기억이 납니다. 이 영화에는 MJ가 언론에 대해 느끼는 미묘한 메시지들과 또한 마약은 나쁘다는 분명한 메시지가 담겨 있습니다.

시각적으로 인상적이지만 이것은 모두 마이클 잭슨에 관한 것이므로 MJ를 조금이라도 좋아하지 않는다면 이 영화를 싫어하고 지루하다고 느낄 것입니다. 어떤 이들은 이 영화 제작에 동의한 MJ를 자기도취자라고 할 수도 있지만 MJ와 대부분의 팬들은 팬들을 위해 만들었다고 말할 것이고, 그것이 사실이라면 정말로 친절한 일입니다.

본격적인 피처 필름 부분이 시작될 때는 약 20분 정도 지속되고 'Smooth Criminal' 시퀀스는 제외됩니다. 조 페시는 심리적으로 전능한 마약 군주로서 설득력 있는 연기를 했습니다. 왜 그가 MJ를 그렇게 죽이고 싶어하는지는 저에게는 이해되지 않습니다. MJ가 그의 계획을 엿들었기 때문일까요? 아니요, 조 페시의 캐릭터는 사람들이 자신이 마약을 공급하는 사람임을 알기를 원했다고 고함을 질렀습니다. 그래서 모르겠네요, 아마도 그는 그저 MJ의 음악이 싫은 것 같습니다.

이 영화에는 MJ가 차나 로봇으로 변하는 등 멋진 것들이 많이 있습니다. 그리고 'Speed Demon' 시퀀스 전체도 그렇습니다. 또한 감독은 아이들과 함께 복잡한 댄스 장면을 연출해야 했기 때문에 성인이 될 때까지 참을성이 필요했을 것입니다.

결론적으로, 이 영화는 MJ를 어느 정도 좋아하는 사람들을 위한 영화입니다(그런 사람이 대부분일 것입니다). 그렇지 않다면 멀리하세요. 이 영화는 건전한 메시지를 전달하려고 시도하고 있고, 아이러니하게도 MJ의 영화에서 가장 친한 친구는 소녀입니다! 마이클 잭슨은 이 지구에 존재한 가장 재능 있는 사람들 중 하나이지만 그가 유죄인지는요? 주의를 기울인 만큼...음, 잘 모르겠습니다. 사람들은 비밀스러운 문을 뒤에 감추고 있을 수 있으니까요. 그는 극도로 착한 사람이지만 어리석은 사람일 수도 있고, 가장 끔찍한 거짓말쟁이일 수도 있습니다. 후자가 아니길 바랍니다.

In [4]:
import re

# <br> html 태그는 replace 함수로 공백으로 변환
review_df['review'] = review_df['review'].str.replace('<br />',' ')

# 파이썬의 정규 표현식 모듈인 re를 이용하여 영어 문자열이 아닌 문자는 모두 공백으로 변환 
review_df['review'] = review_df['review'].apply( lambda x : re.sub("[^a-zA-Z]", " ", x) )


In [5]:
from sklearn.model_selection import train_test_split

class_df = review_df['sentiment']
feature_df = review_df.drop(['id','sentiment'], axis=1, inplace=False)

X_train, X_test, y_train, y_test= train_test_split(feature_df, class_df, test_size=0.3, random_state=156)

X_train.shape, X_test.shape

((17500, 1), (7500, 1))

In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

# 스톱 워드는 English, filtering, ngram은 (1,2)로 설정해 CountVectorization수행. 
# LogisticRegression의 C는 10으로 설정. 
# Pipeline: 머신 러닝 워크플로우를 순차적으로 연결해주는 객체
# solver는 로지스틱 회귀 모델의 최적화 알고리즘, 적은 데이터 처리할 때 사용
# C : 규제(regularization) 강도를 제어하는 하이퍼파라미터, L2규제 작을 수록 강하게 규제를 함
# ngram: 단어를 n개 만큼 확장해서 묶어 처리
pipeline = Pipeline([
    ('cnt_vect', CountVectorizer(stop_words='english', ngram_range=(1,2) )),
    ('lr_clf', LogisticRegression(solver='liblinear', C=10))])

# Pipeline 객체를 이용하여 fit(), predict()로 학습/예측 수행. predict_proba()는 roc_auc때문에 수행.  
pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]

# ROC-AUC: 이진분류기의 성능 (1에 가까울 수록 좋은 성능, 0.5에 가까울 수록 최악성능)
print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test ,pred),
                                         roc_auc_score(y_test, pred_probs)))

예측 정확도는 0.8859, ROC-AUC는 0.9503


In [7]:
# 스톱 워드는 english, filtering, ngram은 (1,2)로 설정해 TF-IDF 벡터화 수행. 
# LogisticRegression의 C는 10으로 설정. 
# ngram_range: 단어를 연속된 2개이 단어도 묶어서 Feature로 활용
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2) )),
    ('lr_clf', LogisticRegression(solver='liblinear', C=10))])

pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test ,pred),
                                         roc_auc_score(y_test, pred_probs)))

예측 정확도는 0.8936, ROC-AUC는 0.9598
